In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.relative_locator import locate_with
from trafilatura import fetch_url, extract
from typing import List

from mongo_handler import setup_mongo_connection, get_reference_list_for_url
from selenium_scraper import setup_driver

In [17]:
mongo_collection = setup_mongo_connection()
f5_references = get_reference_list_for_url("support.f5.com", mongo_collection)
print(f"Found {len(f5_references)} matching reference urls") # [{'_id': 'CVE-2021-XYZ', 'url': 'https://wpscan.com/wtfxyz42'}]
f5_references

Found 932 matching reference urls


[{'_id': 'CVE-2021-22973',
  'url': 'https://support.f5.com/csp/article/K13323323'},
 {'_id': 'CVE-2021-22974',
  'url': 'https://support.f5.com/csp/article/K68652018'},
 {'_id': 'CVE-2021-22975',
  'url': 'https://support.f5.com/csp/article/K21971977'},
 {'_id': 'CVE-2021-22976',
  'url': 'https://support.f5.com/csp/article/K88230177'},
 {'_id': 'CVE-2021-22977',
  'url': 'https://support.f5.com/csp/article/K14693346'},
 {'_id': 'CVE-2021-22978',
  'url': 'https://support.f5.com/csp/article/K87502622'},
 {'_id': 'CVE-2021-22979',
  'url': 'https://support.f5.com/csp/article/K63497634'},
 {'_id': 'CVE-2021-22980',
  'url': 'https://support.f5.com/csp/article/K29282483'},
 {'_id': 'CVE-2021-22981',
  'url': 'https://support.f5.com/csp/article/K09121542'},
 {'_id': 'CVE-2021-22982',
  'url': 'https://support.f5.com/csp/article/K72708443'},
 {'_id': 'CVE-2021-22983',
  'url': 'https://support.f5.com/csp/article/K76518456'},
 {'_id': 'CVE-2021-22984',
  'url': 'https://support.f5.com/csp/a

In [19]:
driver = setup_driver()

In [20]:
#url = 'https://support.f5.com/csp/article/K13323323'
#cve_id = 'CVE-2021-22973'

#url = 'https://support.f5.com/csp/article/K43650115'
#cve_id = 'CVE-2016-0723'

url = 'https://support.f5.com/csp/article/K64346530?utm_source=f5support&amp;utm_medium=RSS' #Bad URL
cve_id = 'CVE-2019-11168'

driver.get(url)

In [23]:
section: WebElement = driver.find_element(By.CSS_SELECTOR, 'div.article-content.ng-binding')
section

<selenium.webdriver.remote.webelement.WebElement (session="9b9b8184fc8853705b677cda02b0e14a", element="caad1a12-5ad5-474f-812e-761aa8e2f647")>

In [24]:
li_elements: List[WebElement] = section.find_elements(By.TAG_NAME, 'li')
li_elements
#li_element: WebElement = section.find_element(By.XPATH, f"//li[contains(text(), '{cve_id}')]")
#li_element

[<selenium.webdriver.remote.webelement.WebElement (session="9b9b8184fc8853705b677cda02b0e14a", element="478ac125-f867-4416-876c-eae96d65845f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9b9b8184fc8853705b677cda02b0e14a", element="c096794b-d7b5-44f8-bf55-44efaeaf47d8")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9b9b8184fc8853705b677cda02b0e14a", element="0407e74e-8fa4-4f27-a4b0-63313c536e6e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9b9b8184fc8853705b677cda02b0e14a", element="b8f22c85-b523-4907-8e18-854bd3f4a01e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9b9b8184fc8853705b677cda02b0e14a", element="9ee582cf-35be-492b-92a8-f793cdf5b047")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9b9b8184fc8853705b677cda02b0e14a", element="3b5be474-2fd1-4a04-a3f3-9fbd1014a55e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="9b9b8184fc8853705b677cda02b0e14a", element="4047ce0b-d456-406f-85b2-02

In [25]:
for li_element in li_elements:
    a_element: WebElement = li_element.find_element(By.TAG_NAME, 'a')
    if a_element and a_element.text == cve_id:
        print('gotem!')

gotem!


In [15]:
text: str = section.text
text = text.replace(f'({cve_id})', '')
text

'Race condition in the tty_ioctl function in drivers/tty/tty_io.c in the Linux kernel through 4.4.1 allows local users to obtain sensitive information from kernel memory or cause a denial of service (use-after-free and system crash) by making a TIOCGETD ioctl call during processing of a TIOCSETD ioctl call. \nImpact\nAuthenticated, local users with Advanced Shell (bash) access for BIG-IP, BIG-IQ, F5 iWorkflow, or Enterprise Manager systems, or console access for Traffix SDC systems, can install a specially crafted program to read very small amounts (4 bytes) of memory or potentially cause a kernel panic.'

In [16]:
text = text.replace('\nImpact\n', '')
text

'Race condition in the tty_ioctl function in drivers/tty/tty_io.c in the Linux kernel through 4.4.1 allows local users to obtain sensitive information from kernel memory or cause a denial of service (use-after-free and system crash) by making a TIOCGETD ioctl call during processing of a TIOCSETD ioctl call. Authenticated, local users with Advanced Shell (bash) access for BIG-IP, BIG-IQ, F5 iWorkflow, or Enterprise Manager systems, or console access for Traffix SDC systems, can install a specially crafted program to read very small amounts (4 bytes) of memory or potentially cause a kernel panic.'

In [17]:
driver.quit()

In [20]:
trafilatura_downloaded = fetch_url(url)
if trafilatura_downloaded is not None:
    trafilatura_result = extract(trafilatura_downloaded)
    print(trafilatura_result)

None
